In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
# q = bq.query(
# """
# select count(*) as num_rows 
# from bigquery-public-data.geo_us_boundaries.counties
# """)
# q.to_dataframe()

In [3]:
#q1
q1 = bq.query(
"""
SELECT area_land_meters, area_water_meters
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = "Dane"
""")
df = q1.to_dataframe()
area_land = df.loc[0][0]
area_water = df.loc[0][1]
q1_ans = area_water/(area_land + area_water) *100
q1_ans

/tmp/ipykernel_261221/2975260206.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  area_land = df.loc[0][0]
/tmp/ipykernel_261221/2975260206.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  area_water = df.loc[0][1]


3.314489915768649

In [4]:
#q2

q2 = bq.query(
"""
SELECT s.state, COUNT(c.state_fips_code) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties c
JOIN bigquery-public-data.geo_us_boundaries.states s
ON c.state_fips_code = s.state_fips_code
GROUP BY s.state
ORDER BY county_count DESC
limit 5
""")
df2 = q2.to_dataframe()
state_county_dict_2 = df2.set_index('state')['county_count'].to_dict()
state_county_dict_2



{'TX': 254, 'GA': 159, 'VA': 133, 'KY': 120, 'MO': 115}

In [5]:
#q3
q1 = bq.query(
"""
SELECT area_land_meters, area_water_meters
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = "Dane"
""", job_config = bigquery.QueryJobConfig(use_query_cache=False))

q2 = bq.query(
"""
SELECT s.state, COUNT(c.state_fips_code) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties c
JOIN bigquery-public-data.geo_us_boundaries.states s
ON c.state_fips_code = s.state_fips_code
GROUP BY s.state
ORDER BY county_count DESC
limit 5
""", job_config = bigquery.QueryJobConfig(use_query_cache=False))

t1 = q1.total_bytes_billed / 1024**4 * 6.25
t2 = q2.total_bytes_billed / 1024**4 * 6.25

dict = {"q1" : t1, "q2" : t2}
dict

{'q1': 5.9604644775390625e-05, 'q2': 0.00011920928955078125}

In [6]:
ds = bigquery.Dataset("cs544-412920.p8")

bq.create_dataset(ds, exists_ok=True)
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")

load = bq.load_table_from_uri("gs://p8_seong/hdma-wi-2021-split.parquet", "cs544-412920.p8.hdma", job_config=config)
load.result()

LoadJob<project=cs544-412920, location=US, id=e9fcbdc8-df74-48e0-975f-7fdd4f118253>

In [7]:
#q4
[ds.dataset_id for ds in bq.list_datasets("cs544-412920")] # PASTE your project name

['p8']

In [8]:
#q5
q5 = bq.query("""
SELECT counties.county_name, COUNT(*) as count
FROM cs544-412920.p8.hdma as hdma
INNER JOIN bigquery-public-data.geo_us_boundaries.counties as counties
ON counties.county_fips_code = hdma.county_code
GROUP BY county_name
ORDER BY count DESC
LIMIT 10
""")

count_5 = {r["county_name"]:r["count"] for r in q5.result()}
count_5

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [9]:
#q6
url = "https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("cs544-412920.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

q6 = bq.query("""
SELECT COUNT(*) as num_app
FROM cs544-412920.p8.applications
WHERE income = 150000
""")

q6.to_dataframe().num_app[0]

10

In [10]:
#q7
q7 = bq.query("""
SELECT counties.county_name, COUNT(*) as num_app
FROM cs544-412920.p8.applications as app
JOIN bigquery-public-data.geo_us_boundaries.counties as counties
ON ST_CONTAINS(counties.county_geom, ST_GEOGPOINT(app.longitude, app.latitude))
WHERE counties.state_fips_code = "55"
GROUP BY counties.county_name
""")

q7.to_dataframe()
count_7 = {row["county_name"]:row["num_app"] for row in q7.result()}
count_7

{'Marinette': 3,
 'Dane': 20,
 'Florence': 1,
 'Outagamie': 2,
 'Sauk': 1,
 'Walworth': 7,
 'Oneida': 3,
 'Barron': 1,
 'Milwaukee': 5,
 'Columbia': 1,
 'Sheboygan': 1,
 'Winnebago': 1,
 'Green Lake': 1,
 'Door': 7,
 'Brown': 3,
 'Douglas': 1,
 'Bayfield': 1,
 'Jefferson': 2,
 'Monroe': 2,
 'Fond du Lac': 1,
 'Juneau': 1,
 'Kewaunee': 1}

In [11]:
ml = bq.query("""
CREATE OR REPLACE MODEL `cs544-412920.p8.lr`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) AS 
SELECT loan_amount,income,loan_term
FROM `cs544-412920.p8.hdma`
""")


In [17]:
#q8
import time


while True:
    eval_list = list(bq.list_models(dataset="cs544-412920.p8"))
    if any("lr" in m.model_id for m in eval_list):
        break
    time.sleep(5)

q8 = bq.query("""
SELECT mean_absolute_error
FROM ML.EVALUATE(MODEL `cs544-412920.p8.lr`, (
SELECT loan_amount, income, loan_term
FROM `cs544-412920.p8.hdma`
WHERE dataset = 'test'
))
""")
df = q8.to_dataframe()

ans_8 = df["mean_absolute_error"][0]
ans_8

77323.856677691

In [13]:
#q9
q9 = bq.query("""
SELECT *
FROM ML.WEIGHTS(MODEL `cs544-412920.p8.lr`)
""")
weights_df = q9.to_dataframe()
loan_term_weight = weights_df.loc[weights_df.iloc[:, 0] == 'loan_term', 'weight']

ans=float(loan_term_weight.iloc[0])
ans



380.2252844067741

In [15]:
#q10
q10 = bq.query("""
SELECT *
FROM ML.PREDICT(MODEL `cs544-412920.p8.lr`, (
SELECT loan_amount,income, 360 AS loan_term
FROM `cs544-412920.p8.applications`
))
""")

df_10 =q10.to_dataframe()
(df_10["predicted_loan_amount"] > df_10["loan_amount"]).mean()

0.42718446601941745